## Pulling in Imagenette

In [1]:
import os
import torchvision
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandAugment(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

imagenette_train = torchvision.datasets.Imagenette(root='./', split='train', size='full', download=True, transform=transform)
DATA_PATH = '../input/edges-data/content_imagenette/'

100%|██████████| 1557161267/1557161267 [01:26<00:00, 18004626.29it/s]


Extracting ./imagenette2.tgz to ./


In [2]:
class_map = {
    'n03028079': 'church',
    'n01440764': 'tench',
    'n03000684': 'chain saw',
    'n03425413': 'gas pump',
    'n02979186': 'casette player',
    'n02102040': 'English springer',
    'n03417042': 'garbage truck',
    'n03394916': 'French horn',
    'n03888257': 'parachute',
    'n03445777': 'golf ball'
}

In [3]:
import cv2

styles = ['content_imagenette/content_imagenette', 'color', 'edges', 'stylized_imagenette/stylized_imagenette']

for style in styles:
    folder_path = f'../input/stylized-data/{style}'
    for folder in os.listdir(folder_path):
        print(f'{style} {folder} has {len(os.listdir(os.path.join(folder_path, folder)))} images ....' )

content_imagenette/content_imagenette chain saw has 50 images ....
content_imagenette/content_imagenette English springer has 50 images ....
content_imagenette/content_imagenette church has 50 images ....
content_imagenette/content_imagenette French horn has 50 images ....
content_imagenette/content_imagenette gas pump has 50 images ....
content_imagenette/content_imagenette golf ball has 50 images ....
content_imagenette/content_imagenette garbage truck has 50 images ....
content_imagenette/content_imagenette cassette player has 50 images ....
content_imagenette/content_imagenette parachute has 50 images ....
content_imagenette/content_imagenette tench has 50 images ....
color chain saw has 50 images ....
color English springer has 50 images ....
color church has 50 images ....
color French horn has 50 images ....
color gas pump has 50 images ....
color golf ball has 50 images ....
color garbage truck has 50 images ....
color cassette player has 50 images ....
color parachute has 50 i

## Applying Canny Filter

In [4]:
# import os
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# def apply_canny_filter(image_path):
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


#     gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     edges = cv2.Canny(gray_image, threshold1=100, threshold2=200)
#     edges_display = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)  # Convert to 3-channel for display
    
#     return edges_display

# def process_images(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)

#     for filename in os.listdir(input_folder):
#         image_path = os.path.join(input_folder, filename)
#         edges_display = apply_canny_filter(image_path)
#         output_path = os.path.join(output_folder, filename)
#         cv2.imwrite(output_path, cv2.cvtColor(edges_display, cv2.COLOR_RGB2BGR))  # Save as BGR
    
#     print(f"Processed and saved: {class_map[input_folder.split('/')[-1]]}")

In [5]:
# import os
# import cv2

# output_root = os.path.join(os.getcwd(), 'edges')
# input_root = DATA_PATH

# # Create output root if it doesn't exist
# if not os.path.exists(output_root):
#     os.mkdir(output_root)

# # Get input folders and corresponding output folders
# input_folders = [os.path.join(input_root, folder) for folder in os.listdir(input_root)]
# output_folders = [os.path.join(output_root, os.path.basename(folder)) for folder in input_folders]

# for in_folder, out_folder in zip(input_folders, output_folders):
#     # Create output subdirectory if it doesn't exist
#     if not os.path.exists(out_folder):
#         os.mkdir(out_folder)

#     for filename in os.listdir(in_folder):
#         image_path = os.path.join(in_folder, filename)
#         edges_display = apply_canny_filter(image_path)

#         # Ensure the output path is valid before saving
#         output_path = os.path.join(out_folder, filename)
#         cv2.imwrite(output_path, cv2.cvtColor(edges_display, cv2.COLOR_RGB2BGR))

#     print(f"Processed and saved: {os.path.basename(in_folder)} at {out_folder}")

## Finetuning and Inference

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models

torch.manual_seed(42)

model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=10)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 172MB/s] 


In [7]:
val_dir = 'imagenette2/val'
train_dir = 'imagenette2/train'
folders = os.listdir(train_dir)

for folder in folders:
    if folder in class_map:
        old_folder = os.path.join(train_dir, folder)
        new_folder = os.path.join(train_dir, class_map[folder])
        os.rename(old_folder, new_folder)
        print(f'Renamed {folder} to {class_map[folder]}')

folders = os.listdir(val_dir)

for folder in folders:
    if folder in class_map:
        old_folder = os.path.join(val_dir, folder)
        new_folder = os.path.join(val_dir, class_map[folder])
        os.rename(old_folder, new_folder)
        print(f'Renamed {folder} to {class_map[folder]}')

Renamed n03394916 to French horn
Renamed n02102040 to English springer
Renamed n03000684 to chain saw
Renamed n03425413 to gas pump
Renamed n03445777 to golf ball
Renamed n01440764 to tench
Renamed n03028079 to church
Renamed n02979186 to casette player
Renamed n03888257 to parachute
Renamed n03417042 to garbage truck
Renamed n03394916 to French horn
Renamed n02102040 to English springer
Renamed n03000684 to chain saw
Renamed n03425413 to gas pump
Renamed n03445777 to golf ball
Renamed n01440764 to tench
Renamed n03028079 to church
Renamed n02979186 to casette player
Renamed n03888257 to parachute
Renamed n03417042 to garbage truck


In [8]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_data_path = 'imagenette2/train'
normal_path = '../input/stylized-data/content_imagenette/content_imagenette'
edge_path = '../input/stylized-data/edges'
color_path = '../input/stylized-data/color'
shape_path = '../input/stylized-data/stylized_imagenette/stylized_imagenette'


# Define the data transformations (you can adjust these as needed)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandAugment(),
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

train_dataset = datasets.ImageFolder(train_data_path, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder('imagenette2/val', transform=data_transforms['val'])

val_dataset = [datasets.ImageFolder(normal_path, transform=data_transforms['val']),
               datasets.ImageFolder(edge_path, transform=data_transforms['val']),
               datasets.ImageFolder(color_path, transform=data_transforms['val']),
               datasets.ImageFolder(shape_path, transform=data_transforms['val']),
              ]



train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
normal_loader = DataLoader(val_dataset[0] , batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
edge_loader = DataLoader(val_dataset[1] , batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
color_loader = DataLoader(val_dataset[2] , batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
shape_loader = DataLoader(val_dataset[3] , batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset[0])}')

Training samples: 9469
Validation samples: 500


In [9]:
def train_step(model, dataloader, criterion, optimizer, device, name):
    '''Train for one epoch'''
    
    model.train()

    train_loss = 0.0
    train_acc = 0.0

    for i, data in enumerate(dataloader):
        if name == 'PACS':
            X = data['images']
            y = torch.squeeze(data['labels'])

            X = X.to(device)
            y = y.to(device)

        else:
            X, y = data[0].to(device), data[1].to(device)
        
        logits = model(X)
        loss = criterion(logits, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred = torch.argmax(logits.detach(), dim=1)
        train_acc += (y_pred == y).sum().item() / len(y)

        # Print dynamic progress on the same line using \r
        print(f'\rTraining: [{i+1}/{len(dataloader)}] '
              f'Loss: {train_loss / (i + 1):.4f} '
              f'Acc: {train_acc / (i + 1):.4f}', end='')

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    
    # Move to the next line after the loop is done
    print()  
    
    return train_loss, train_acc

@torch.inference_mode()
def eval_step(model, dataloader, criterion, device, name):
    '''Evaluate the model'''
    
    model.eval()

    eval_loss = 0.0
    eval_acc = 0.0

    for i, data in enumerate(dataloader):
        
        if name in ['PACS']:
            X = data['images']
            y = torch.squeeze(data['labels'])

            X = X.to(device)
            y = y.to(device)

        else:
            X, y = data[0].to(device), data[1].to(device)
        
        logits = model(X)
        loss = criterion(logits, y)
        eval_loss += loss.item()

        y_pred = torch.argmax(logits.detach(), dim=1)
        eval_acc += (y_pred == y).sum().item() / len(y)

        # Print dynamic progress on the same line using \r
        print(f'\rEvaluation: [{i+1}/{len(dataloader)}] '
              f'Loss: {eval_loss / (i + 1):.4f} '
              f'Acc: {eval_acc / (i + 1):.4f}', end='')

    eval_loss = eval_loss / len(dataloader)
    eval_acc = eval_acc / len(dataloader)
    
    # Move to the next line after the loop is done
    print()  
    
    return eval_loss, eval_acc

In [16]:
import torch.optim as optim
import csv
from tqdm import tqdm 
import time as time

out_dir = 'output'
os.makedirs(out_dir, exist_ok=True)
epochs = 3
lr = 3e-4

loaders = [normal_loader, edge_loader, color_loader, shape_loader]
names = ['normal', 'edge', 'color', 'stylized']

train_dl = train_loader
# test_dl = val_loader
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

bar_format = '{l_bar}{bar} | Epoch: {n_fmt}/{total_fmt} | Time: {elapsed} < {remaining} | {rate_fmt}'
model.to(device)

for epoch in tqdm(range(epochs), desc="Epochs", bar_format=bar_format, leave=True):
    start_time = time.time()  # Track the start time of the epoch

    train_loss, train_acc = train_step(model, train_dl, criterion, optimizer, device, 'custom')
    val_loss, val_acc = eval_step(model, loaders[0], criterion, device, 'custom')

    epoch_duration = time.time() - start_time

    tqdm.write(f"============ Epoch {epoch + 1} --> Train Acc: {train_acc:.4f} || Val Acc: {val_acc:.4f} || Time: {epoch_duration:.2f} s ============\n")

In [17]:
names = ['normal', 'edge', 'color', 'stylized', 'test']
loaders.append(DataLoader(test_dataset , batch_size=BATCH_SIZE, shuffle=False, num_workers=2))
normal_loss, normal_acc = eval_step(model, loaders[0], criterion, device, 'custom')

for name, val_loader in zip(names[1:], loaders[1:]): 
    val_loss, val_acc = eval_step(model, val_loader, criterion, device, 'custom')
    print(f'\n=========== {name} accuracy: {val_acc*100:.3f}% || normal accuracy: {normal_acc*100:.3f}% ===========\n')

Evaluation: [16/16] Loss: 0.0899 Acc: 0.9805
Evaluation: [16/16] Loss: 1.7781 Acc: 0.4094

=========== edge accuracy: 40.938% || normal accuracy: 98.047% ===========

Evaluation: [16/16] Loss: 0.3008 Acc: 0.9207

=========== color accuracy: 92.070% || normal accuracy: 98.047% ===========

Evaluation: [16/16] Loss: 1.3855 Acc: 0.5711

=========== stylized accuracy: 57.109% || normal accuracy: 98.047% ===========

Evaluation: [123/123] Loss: 0.0713 Acc: 0.9878

=========== test accuracy: 98.780% || normal accuracy: 98.047% ===========



## Testing the Scripts

In [5]:
import shutil 

REPO_ROOT = '../input/atml-repo'
dest_path = './repo'

# Make sure the destination directory exists
os.makedirs(dest_path, exist_ok=True)

# Copy the entire directory from REPO_ROOT to dest_path
shutil.copytree(REPO_ROOT, dest_path, dirs_exist_ok=True)

print(f"Folder copied from {REPO_ROOT} to {dest_path}")

Folder copied from ../input/atml-repo to ./repo


In [6]:
%%writefile './repo/custom_infer.py'
import argparse
import time 
import csv
import matplotlib.pyplot as plt 
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from model import *
from data import *
from utils import *

def train_step(model, dataloader, criterion, optimizer, device):
    '''Train for one epoch'''
    
    model.train()

    train_loss = 0.0
    train_acc = 0.0

    for i, data in enumerate(dataloader):

        X, y = data[0].to(device), data[1].to(device)
        
        logits = model(X)
        loss = criterion(logits, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred = torch.argmax(logits.detach(), dim=1)
        train_acc += (y_pred == y).sum().item() / len(y)

        # Print dynamic progress on the same line using \r
        print(f'\rTraining: [{i+1}/{len(dataloader)}] '
              f'Loss: {train_loss / (i + 1):.4f} '
              f'Acc: {train_acc / (i + 1):.4f}', end='')

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    
    # Move to the next line after the loop is done
    print()  
    
    return train_loss, train_acc

@torch.inference_mode()
def eval_step(model, dataloader, criterion, device):
    '''Evaluate the model'''
    
    model.eval()

    eval_loss = 0.0
    eval_acc = 0.0

    for i, data in enumerate(dataloader):

        X, y = data[0].to(device), data[1].to(device)
        
        logits = model(X)
        loss = criterion(logits, y)
        eval_loss += loss.item()

        y_pred = torch.argmax(logits.detach(), dim=1)
        eval_acc += (y_pred == y).sum().item() / len(y)

        # Print dynamic progress on the same line using \r
        print(f'\rEvaluation: [{i+1}/{len(dataloader)}] '
              f'Loss: {eval_loss / (i + 1):.4f} '
              f'Acc: {eval_acc / (i + 1):.4f}', end='')

    eval_loss = eval_loss / len(dataloader)
    eval_acc = eval_acc / len(dataloader)
    
    # Move to the next line after the loop is done
    print()  
    
    return eval_loss, eval_acc

if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description='Finetune a pretrained ResNet!')
    parser.add_argument('--model_name', type=str, default='resnet-34', help='Choose from [resnet-34, vit or clip]. Default is resnet-34.')
    parser.add_argument('--save_model', type=bool, help='Specify whether the trained model must be saved or not. Will be saved in {output_dir}/models.', default=True)
    parser.add_argument('--out_dir', type=str, default='./output', help='Path to the directory where training log and model will be saved.')
    parser.add_argument('--train_path', type=str, help='Path to training data.')
    parser.add_argument('--val_path', type=str, help='Path to validation data.')
    parser.add_argument('--lr', type=float, help='Learning Rate. Default is 1e-4', default=1e-4)
    parser.add_argument('--batch_size', type=int, help='Batch size. Default is 32.', default=32)
    parser.add_argument('--epochs', type=int, help='Number of fine-tuning epochs. Default is 15', default=15)
    
    args = parser.parse_args()

    set_seed()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    batch_size = args.batch_size
    num_workers = 2

    os.makedirs(f'{args.out_dir}/model', exist_ok=True)
    os.makedirs(f'{args.out_dir}/log', exist_ok=True)

    train_ds, test_ds = get_custom_data(args.train_path, args.val_path)

    train_dl = get_dataloader(train_ds, batch_size, True, num_workers)
    test_dl = get_dataloader(test_ds, batch_size, False, num_workers)

    num_classes = len(train_ds.classes)

    model = load_resnet_ft(args.model_name, 'classifier')
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)

    print(f'\nFinetuning {args.model_name} with {count_parameters(model) * 1e-6:.3f}M params for {args.epochs} epochs on {device}...\n')

    best_loss = float('inf')
    bar_format = '{l_bar}{bar} | Epoch: {n_fmt}/{total_fmt} | Time: {elapsed} < {remaining} | {rate_fmt}'

    with open(os.path.join(args.out_dir, 'log/run.csv'), 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['epoch', 'train_loss', 'train_acc', 'test_loss', 'test_acc'])

        for epoch in tqdm(range(args.epochs), desc="Epochs", bar_format=bar_format, leave=True):
            start_time = time.time()  # Track the start time of the epoch

            train_loss, train_acc = train_step(model, train_dl, criterion, optimizer, device)
            test_loss, test_acc = eval_step(model, test_dl, criterion, device)
            
            if test_loss < best_loss:
                best_loss = test_loss
                if args.save_model:
                    torch.save(model.state_dict(), os.path.join(args.out_dir, 'model/best.pth'))

            writer.writerow([epoch + 1, train_loss, train_acc, test_loss, test_acc])
            
            # Calculate epoch duration
            epoch_duration = time.time() - start_time
            
            # Use tqdm.write to print epoch summary with duration
            tqdm.write(f"============ Epoch {epoch + 1} --> Train Acc: {train_acc:.4f} || Test Acc: {test_acc:.4f} || Time: {epoch_duration:.2f} s ============\n")
            
    # Save this model at the end of run (commented out for)
    if args.save_model:
        torch.save(model.state_dict(), os.path.join(args.out_dir, 'model/final.pth'))

    df = pd.read_csv(f'{args.out_dir}/log/run.csv')
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Plot Train Accuracy and Test Accuracy
    ax[0].plot(df['train_acc'], label='Train Accuracy', color='orange', marker='o', linewidth=2)
    ax[0].plot(df['test_acc'], label='Test Accuracy', color='blue', marker='o', linewidth=2)
    ax[0].set_title('Train vs Test Accuracy')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Accuracy')
    ax[0].legend()
    ax[0].grid(True)

    # Plot Train Loss and Test Loss
    ax[1].plot(df['train_loss'], label='Train Loss', color='orange', marker='o', linewidth=2)
    ax[1].plot(df['test_loss'], label='Test Loss', color='blue', marker='o', linewidth=2)
    ax[1].set_title('Train vs Test Loss')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Loss')
    ax[1].legend()
    ax[1].grid(True)

    plt.tight_layout()
    plt.savefig(f'{args.out_dir}/log_plot.png')
    plt.show()

Overwriting ./repo/custom_infer.py


In [7]:
train_dir = 'imagenette2/train'

for folder in os.listdir(train_dir):
    if folder in class_map:
        old_folder = os.path.join(train_dir, folder)
        new_folder = os.path.join(train_dir, class_map[folder])
        os.rename(old_folder, new_folder)
        print(f'Renamed {folder} to {class_map[folder]}')

Renamed n03000684 to Chainsaw
Renamed n03888257 to Parachute
Renamed n03417042 to Garbage Truck
Renamed n03445777 to Golf Ball
Renamed n01440764 to Tench
Renamed n03028079 to Church
Renamed n02979186 to Casette Player
Renamed n03425413 to Gas Pump
Renamed n03394916 to French Horn
Renamed n02102040 to English Springer


In [8]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=27ac91963e86e2e953daee81ef453ca7ec8f916d389d80ee0e30d6272dd8746c
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [9]:
train_data_path = 'imagenette2/train'
val_data_path = 'edges'

os.system(f'python repo/custom_infer.py --epochs 3 --out_dir script_output --lr 3e-4 --train_path {train_data_path} --val_path {val_data_path}')

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 176MB/s]



Freezing non-classifier head weights ....


Finetuning resnet-34 with 0.513M params for 3 epochs on cuda...



Epochs:   0%|           | Epoch: 0/3 | Time: 00:00 < ? | ?it/s

Training: [296/296] Loss: 3.5074 Acc: 0.3403


Epochs:  33%|███▎       | Epoch: 1/3 | Time: 00:52 < 01:45 | 52.99s/it

...

Epochs:  67%|██████▋    | Epoch: 2/3 | Time: 01:45 < 00:52 | 52.67s/it

Evaluation: [16/16] Loss: 3.4283 Acc: 0.1562
============ Epoch 2 --> Train Acc: 0.8442 || Test Acc: 0.1562 || Time: 52.44 s ============

Training: [296/296] Loss: 0.2812 Acc: 0.9217


Epochs: 100%|██████████ | Epoch: 3/3 | Time: 02:38 < 00:00 | 52.69s/it


Evaluation: [16/16] Loss: 3.5696 Acc: 0.1777
============ Epoch 3 --> Train Acc: 0.9217 || Test Acc: 0.1777 || Time: 52.63 s ============



0